# SpaceX Launch Record Dashboard create

## Objectives:
* Import required libraries
* Download a skeleton dashboard application and dataset
* Add a Launch Site Drop-down Input Component
* Add a callback function to render success-pie-chart based on selected site dropdown
* Add a Range Slider to Select Payload
* Add a callback function to render the success-payload-scatter-chart scatter plot
* run the app

### Import required libraries

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output


### Download a skeleton dashboard application and dataset

In [2]:
# Create a dash application
app = dash.Dash(__name__)

In [3]:
URL = "data_files/spacex_launch_geo.csv"

In [4]:
spacex_df = pd.read_csv(URL)
spacex_df.head(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
5,7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.0,GTO,SES,No attempt,0,28.562302,-80.577356
6,8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.0,GTO,Thaicom,No attempt,0,28.562302,-80.577356
7,9,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296.0,LEO (ISS),NASA (CRS),Controlled (ocean),0,28.562302,-80.577356
8,10,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316.0,LEO,Orbcomm,Controlled (ocean),0,28.562302,-80.577356
9,11,2014-08-05,8:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535.0,GTO,AsiaSat,No attempt,0,28.562302,-80.577356


### Adding a Launch Site dropdown Input component

`dcc.Dropdown(id='site-dropdown',...)` input with following attributes:
* `id` attribute with value `site-dropdown`
* `options` attribute is a list of dict-like option objects (with `label` and `value` attributes). You can set
the `label` and `value` all to be the launch site names in the `spacex_df`
* and you need to include the default All option. e.g.,
  `options` =[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]
* `value` attribute with default `dropdown` value to be `ALL` meaning all sites are selected
* `placeholder` attribute to show a text description about this input area,such as `Select a Launch Site here`
* `searchable` attribute to be True so we can enter keywords to search launch sites

In [5]:
# spacex_df['Launch Site'].value_counts()

In [6]:
dropdown = dcc.Dropdown(id = 'site-dropdown',
            options= [ {'label': 'All sites','value': 'ALL'},
                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
            ],
            value = 'ALL',
            placeholder= 'Select a Launch Site here',
            searchable= True
            )

### Add a slider to select payload range

In [7]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [8]:
range_slider = dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,
                value=[min_payload,max_payload],
                marks={0: '0', 2500:'2500',5000:'5000',
                7500:'7500', 10000: '10000'})

### Make the application Layout

In [9]:
app.layout = html.Div([html.H1('SpaceX Launch Record Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                       html.Div([dropdown,
                                 html.Br(),
                                 # Addding a pie chart to show the total successful launches count for all sites
                                 # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                 html.Div(dcc.Graph(id='success-pie-chart')),
                                 html.Br(),
                                 html.P("Payload range (Kg):"),
                                 html.Div(range_slider),
                                 # Add a scatter chart to show the correlation between payload and launch success
                                 html.Div(dcc.Graph(id='success-payload-scatter-chart')),      
                                ])
                      ])


### Add a callback function to render `success-pie-chart` based on selected site dropdown

The general idea of this `callback` function is to get the selected launch site from `site-dropdown` and render
a pie chart visualizing launch success counts.

Dash `callback` function is a type of Python function which will be automatically called by
Dash whenever receiving an input component updates, such as a click or dropdown selecting event

In [10]:
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(site_dropdown):
    if site_dropdown == 'ALL':
        filtered_df = spacex_df
        figure = px.pie(filtered_df, values='class', 
                    names= 'Launch Site', 
                    title= 'Total success Launches by sites')
        return figure
    else:
        specific_df= spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_df = pd.DataFrame(columns= ['Launch type','count'],
                                   data = [["Success",specific_df.loc[specific_df['class'] == 1].shape[0]],
                                                           ["Failure",specific_df.loc[specific_df['class'] == 0].shape[0]]])
        figure = px.pie(filtered_df,
                    values='count',
                    names='Launch type',
                    title= 'Total success Launches by {} sites'.format(site_dropdown))
        return figure
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])
def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = spacex_df [(spacex_df['Payload Mass (kg)']>=payload_slider[0]) 
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class", 
                                 color="Booster Version Category",size = 'Payload Mass (kg)' )
        return scatterplot
    else:
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class",
                                 color="Booster Version Category",size = 'Payload Mass (kg)')
        return scatterplot

### Run the App

In [11]:
# Run the app
# if __name__ == '__main__':
#     app.run_server()